# import声明

In [ ]:
import pandas as pd
import numpy as np
import faiss
from tqdm import tqdm
import time

#  1.读取train数据

In [17]:
print('start')

start = time.time()
train_behavior = pd.read_csv('./data/train/behaviors.tsv',sep ='\t',header=None)
###处理 behaviors.tsv 行为数据
train_behavior.columns = ['index','id','timeStamp','clickHistory','impressionLog']
impressionLog = train_behavior['clickHistory'].str.split(' ', expand=True).stack().reset_index(level = 1,drop = True)

train_data = pd.DataFrame()
train_data= train_behavior[['id']].join(pd.DataFrame(impressionLog,columns=['clickHistory']))

train_data.dropna(axis=0, how='any', inplace=True)
print(train_data.shape[0])
print('time is : ',time.time() - start)


start
5107639
time is :  24.138913869857788


# 2.读取test数据

In [ ]:
print('start')

start = time.time()
test_behavior = pd.read_csv('./data/test/behaviors.tsv',sep ='\t',header=None)
###处理 behaviors.tsv 行为数据
test_behavior.columns = ['index','id','timeStamp','clickHistory','impressionLog']
impressionLog = test_behavior['clickHistory'].str.split(' ', expand=True).stack().reset_index(level = 1,drop = True)

test_data = pd.DataFrame()
test_data= test_behavior[['id']].join(pd.DataFrame(impressionLog,columns=['clickHistory']))

test_data.dropna(axis=0, how='any', inplace=True)
print(test_data.shape[0])
print('time is : ',time.time() - start)

# 3.合并train和test

In [19]:
all_data = pd.concat([train_data,test_data],axis=0).reset_index().drop('index',1)


# 4.读取用户ID和历史点击新闻ID

In [20]:

## 用户id映射
id_numbers = all_data['id'].drop_duplicates().shape[0]
pre_id = all_data['id'].drop_duplicates().tolist()
now_id = {}
for index in tqdm(range(id_numbers)):
    now_id[pre_id[index]] = index+1
curr_id = all_data['id'].tolist()
for index in tqdm(range(len(curr_id))):
    curr_id[index] = now_id[curr_id[index]]
    
## 历史点击新闻ID映射    
his_numbers = all_data['clickHistory'].drop_duplicates().shape[0]
pre_his = all_data['clickHistory'].drop_duplicates().tolist()
now_his = {}
for index in tqdm(range(his_numbers)):
    now_his[pre_his[index]] = id_numbers+index+1
curr_his = all_data['clickHistory'].tolist()
for index in tqdm(range(len(curr_id))):
    curr_his[index] = now_his[curr_his[index]] 

## 用户id反向映射
index_user = {}
index_user[-1] = 'None'
for index in tqdm(range(id_numbers)):
    index_user[index] = pre_id[index]
    
##数据存储
data = pd.DataFrame()
data['id'] = curr_id
data['his'] = curr_his
data.to_csv('./data/id_news.csv',index=0,header=0,sep='\t')

100%|██████████| 7470153/7470153 [00:05<00:00, 1393514.21it/s]


# 5. 通过node2vec得到用户embedding


In [ ]:
!pecanpy --input ./data/id_news.csv  --output ./data/news.emb --mode SparseOTF --workers 44


# 6.用户ID获取

In [ ]:

news_emb = pd.read_csv('./data/news.emb',header=None)
data = news_emb[0].str.split(' ',expand=True)
new_data = pd.DataFrame()
new_data['user'] = data[0]
new_data['emb'] = data[[x for x in range(1,129)]].values.tolist()
new_data = new_data[new_data['user'].astype(int)<91936]
new_data['user'] = new_data['user'].astype(int)
new_data.sort_values('user',inplace=True)
new_data.to_csv('./data/user_emb.csv',index=False)

# 7.计算用户相似度

In [7]:

new_data = pd.read_csv('./data/user_emb.csv')
emb = new_data['emb'].apply(lambda x : [float(y) for y in eval(x)]).values.tolist()
emb = np.array(emb).astype('float32')
d = 128
nlist = 100
quantizer = faiss.IndexFlatL2(d)  # the other index
gpu_index = faiss.index_cpu_to_all_gpus(quantizer) # build the index
index = faiss.IndexIVFFlat(gpu_index,d,nlist,faiss.METRIC_L2)
index.train(emb)
index.add(emb)

k = 5
D,I = index.search(emb,k)



# 8.相似用户存储

In [15]:
neighbor = pd.DataFrame()
user = [x for x in range(91935)]
nearst = []
arr = I.tolist()
for index in range(I.shape[0]):
    nearst.append(arr[index][1:])
neighbor['user'] = user
neighbor['nearst'] = nearst
neighbor['user'] =  neighbor['user'].apply(lambda x : index_user[int(x)])
neighbor['nearst'] =  neighbor['nearst'].apply(lambda x : [index_user[int(y)] for y in x])
neighbor.to_csv('./data/nearst_4.csv',index=False)